In [4]:
# imports
import pydicom
import io
from skimage.measure import label, regionprops
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
def get_bounding_boxes(mask, label_value):
    labeled_mask = label(mask == label_value)
    regions = regionprops(labeled_mask)
    boxes = []
    for region in regions:
        min_row, min_col, max_row, max_col = region.bbox
        boxes.append((min_row, min_col, max_row, max_col))
    return boxes

def draw_bounding_boxes(image, boxes, margin=5, thickness=2):
    img_bbox = Image.fromarray((255 * image).astype('uint8')).convert('RGB')
    draw = ImageDraw.Draw(img_bbox)
    for box in boxes:
        top, left, bottom, right = box
        top = max(0, top - margin)
        left = max(0, left - margin)
        bottom = min(image.shape[0], bottom + margin)
        right = min(image.shape[1], right + margin)
        for i in range(thickness):
            draw.rectangle(
                [left + i, top + i, right - i, bottom - i],
                outline=(255, 0, 0)
            )
    del draw
    return np.asarray(img_bbox)

def convert_mask_to_bounding_box(img, seg_mask):
    # Decode the DICOM file data
    ds = pydicom.dcmread(io.BytesIO(img.read()))
    image = ds.pixel_array.astype(float)

    # Decode the segmentation file data
    seg_ds = pydicom.dcmread(io.BytesIO(seg_mask.read()))
    mask = seg_ds.pixel_array

    # Get bounding boxes for regions labeled as 2
    boxes = get_bounding_boxes(mask, 2)

    # Draw bounding boxes on the image
    return draw_bounding_boxes(image, boxes)

In [ ]:
# files
dicom = "path/to/dicom.dcm"
seg_dicom = "path/to/seg_dicom.dcm"

bounding_box_image = convert_mask_to_bounding_box(dicom, seg_dicom)